Details project at https://github.com/holedev/improve-cifar10

## Import

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def get_default_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Load Datasets

In [3]:
batch_size = 20

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:11<00:00, 14.5MB/s]


## Define Model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HeheNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [5]:
device = get_default_device()
net = HeheNet().to(device)

## Optimize

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training

In [7]:
num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader)
    print(f"📘 Epoch [{epoch + 1:3d}/{num_epochs}] - Loss: {epoch_loss:.4f}")

📘 Epoch [  1/100] - Loss: 1.5370
📘 Epoch [  2/100] - Loss: 1.1897
📘 Epoch [  3/100] - Loss: 1.0365
📘 Epoch [  4/100] - Loss: 0.9432
📘 Epoch [  5/100] - Loss: 0.8762
📘 Epoch [  6/100] - Loss: 0.8163
📘 Epoch [  7/100] - Loss: 0.7746
📘 Epoch [  8/100] - Loss: 0.7443
📘 Epoch [  9/100] - Loss: 0.7086
📘 Epoch [ 10/100] - Loss: 0.6815
📘 Epoch [ 11/100] - Loss: 0.6571
📘 Epoch [ 12/100] - Loss: 0.6392
📘 Epoch [ 13/100] - Loss: 0.6145
📘 Epoch [ 14/100] - Loss: 0.5972
📘 Epoch [ 15/100] - Loss: 0.5802
📘 Epoch [ 16/100] - Loss: 0.5650
📘 Epoch [ 17/100] - Loss: 0.5485
📘 Epoch [ 18/100] - Loss: 0.5330
📘 Epoch [ 19/100] - Loss: 0.5240
📘 Epoch [ 20/100] - Loss: 0.5078
📘 Epoch [ 21/100] - Loss: 0.5000
📘 Epoch [ 22/100] - Loss: 0.4828
📘 Epoch [ 23/100] - Loss: 0.4695
📘 Epoch [ 24/100] - Loss: 0.4605
📘 Epoch [ 25/100] - Loss: 0.4513
📘 Epoch [ 26/100] - Loss: 0.4430
📘 Epoch [ 27/100] - Loss: 0.4286
📘 Epoch [ 28/100] - Loss: 0.4271
📘 Epoch [ 29/100] - Loss: 0.4154
📘 Epoch [ 30/100] - Loss: 0.4071
📘 Epoch [ 

## Testing

In [8]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predictions = torch.max(outputs, 1)

        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

        total += labels.size(0)
        correct += (predictions == labels).sum().item()

for classname in classes:
    accuracy = 100 * correct_pred[classname] / total_pred[classname]
    print(f'=== Accuracy for class: {classname:5s} is {accuracy:.1f} % ===')

print("=" * 50)
print(f'🎯 Overall accuracy on 10000 test images: {100 * correct / total:.2f} %')
print("=" * 50)

=== Accuracy for class: plane is 92.6 % ===
=== Accuracy for class: car   is 96.2 % ===
=== Accuracy for class: bird  is 83.9 % ===
=== Accuracy for class: cat   is 83.4 % ===
=== Accuracy for class: deer  is 89.2 % ===
=== Accuracy for class: dog   is 84.7 % ===
=== Accuracy for class: frog  is 93.1 % ===
=== Accuracy for class: horse is 92.8 % ===
=== Accuracy for class: ship  is 92.8 % ===
=== Accuracy for class: truck is 92.5 % ===
🎯 Overall accuracy on 10000 test images: 90.12 %
